### 5.1.1

1. Align against reference Capsella rubella
   1. Arabidopsis has only 5 chromosomes and smaller genome --> therefor not taking that
   2. Neslia raw data is not accessible
2. Monte Gargano (Italy) reference strain. Reads were collected with Roche 454 XLR and Illumina GAIIx machines. Sanger sequencing
   1. Assembled with Arachne and produced 1859 scaffolds
   2. 

### 5.1.2

1. Blast is to slow for short read mapping against the whole genome. Read mapping is finding the genes, where the scaffolds are coming from. Aligning the scaffolds against a reference.
2. 

### 5.1.3

1. downloaded
2. Genomic data in gff format, which describes genes and other features of DNA, RNA and protein sequences
   1. Protein data in faa format --> fasta
   2. rna data in fna format --> predicted 
   3. cds from genomic --> fasta file, which contain the protein coding sequences. Starts with ATG and ends with stop codon (coding sequence CDS)
   4. GCF*.fna --> scaffolds
3. GCF*.fna file

### 5.1.4

1. used minimap2 instead --> ok gives weird file
   1. `bwa index ~/hts-exercises/5-1/crubellaData/ncbi_dataset/data/GCF_000375325.1/GCF_000375325.1_Caprub1_0_genomic.fna `
      1. Better way of indexing: `mkdir ~/hts-exercises/5-1/crubellaData/ncbi_dataset/data/idx && ln -s ~/hts-exercises/5-1/crubellaData/ncbi_dataset/data/GCF_000375325.1/GCF_000375325.1_Caprub1_0_genomic.fna ~/hts-exercises/5-1/crubellaData/ncbi_dataset/data/idx/ref && bwa index ~/hts-exercises/5-1/crubellaData/ncbi_dataset/data/idx/ref`
      2. Index is then in a well known folder and does not pollute the original data
   2. `bwa mem ./crubellaData/ncbi_dataset/data/idx/ref ~/../data/Capsella-sequencing/DNA-seq/subsets-phred33/Cru_*.fq.gz > readmapping.sam`
      1. First comes the header with @SQ   SN:NW_006...after that the SAM entries, see below
   3. `samtools view -hSbo readmapping.bam readmapping.sam`
   4. `samtools sort readmapping.bam -o readmapping.sorted`
   5. `samtools index readmapping.sorted`
   6. `samtools tview readmapping.sorted ~/hts-exercises/5-1/crubellaData/ncbi_dataset/data/GCF_000375325.1/GCF_000375325.1_Caprub1_0_genomic.fna`
2. Done in readmapping.sam
3. Sam file --> Position 1368244 in the reference file for gene NW_006238923.1

```
      @SQ	SN:NW_006238154.1	LN:203
@PG	ID:bwa	PN:bwa	VN:0.7.15-r1140	CL:bwa mem ./crubellaData/ncbi_dataset/data/idx/ref /mnt/silo/hts2024/alehmann/../data/Capsella-sequencing/DNA-seq/subsets-phred33/Cru_1.fq.gz /mnt/silo/hts2024/alehmann/../data/Capsella-sequencing/DNA-seq/subsets-phred33/Cru_2.fq.gz

N73019:2:1:1147:14001#0	83	NW_006238923.1	1368454	60	120M	=	1368244	-330	TGGAATGCTCTTCCTCTTGAGACTCCTCCATCGACATTACAGACATATTTAAATGTTTTGCCCACCATTGGCTTACCTAGAGGAAGAAGAGTGAGAAAAGAAAAAACAACAAGACTACTA	##########?)?A,A<<A-AAA+??-@:?4??>=5:5A=DDD:ACC5CA4B==-?:.5=9)9*,5:B=.-AD-DDD-?D:CAC=BD=D=ACAC=3B@BB<B>DBB?@DDB;;@===.>:	NM:i:1	MD:Z:61A58	AS:i:115	XS:i:20

N73019:2:1:1147:14001#0	163	NW_006238923.1	1368244	60	120M	=	1368454	330	GAAACAAACTCATGAACTGTAACATCTGAGGCCAGATCACTCAAGAAAAACGCTTTTCCAATAGCAGTAAAACTTTAGGCAGAACAGCGAAGATAATCCAAAAGACGCTGCATCACACTG	B5.-1?A=;5:CB=:C=5-=@;@#################################################################################################	NM:i:10	MD:Z:10T13G8T14C1T1T22G8A20A11A2	AS:i:72	XS:i:0
```
 
```
Col	Field	Type	Brief description
1	QNAME	String	Query template NAME
2	FLAG	Int	bitwise FLAG
3	RNAME	String	References sequence NAME
4	POS	Int	1- based leftmost mapping POSition
5	MAPQ	Int	MAPping Quality
6	CIGAR	String	CIGAR string --> Compressed representation of alignment. For example indicates Matches, deletions, insertions.
7	RNEXT	String	Ref. name of the mate/next read
8	PNEXT	Int	Position of the mate/next read
9	TLEN	Int	observed Template LENgth
10	SEQ	String	segment SEQuence
11	QUAL	String	ASCII of Phred-scaled base QUALity+33
```

#### What are we doing here:

- Overview: DNA -> shearing -> fragments -> size slection on the gel -> size selected fragments
- -> ligate technical sequences (adapters, primers, indexes)
- pppppppp_primer1_NNNNNNNNNNNNNNNNNNNNNNNNNNN_primer2_ppppppp --> the whole thing in between is the template or fragment
- segment1 and segment2 are the reads from the fragment left and right

### 5.1.5

1. SamTools is a set of utilities for post processing short DNA sequence read alignments in SAM, BAM and CRAM formats
   1. It consumes outputs by short read aligners like bwa
   2. SAM are human readable and BAM are their binary equivalents. CRAm is a colun oriented binary container format
   3. view command can select and filter data. sort sorts a BAM file based on the position in the reference
   4. index creates a new index file. Output is *.sam.sai or *.bam.bai
   5. tview is an interactive ascii based viewer to visualize alignments
2. Done
3. See above
4. See 5.1.4
5. `~/libs/samtools-1.19.2/misc/plot-bamstats -p ./output/output ./readmapping.stats`
   1. Or: `samtools flagstat readmapping.sorted` --> see output below
   2. `samtools idxstats readmapping.sorted | less` --> Identifier of the sequence in the reference, sequence length, # mapped read-segments, # unmapped read-segments. unmapped read-segments are paired reads, where one pair could be aligned, but not the other one
6. 0.6% are unmapped see below, `samtools view -f4 readmapping.sam` f means include flags, F means exclude flags
   
Samtools Flags:

```
Flags:
        0x1     PAIRED        .. paired-end (or multiple-segment) sequencing technology
        0x2     PROPER_PAIR   .. each segment properly aligned according to the aligner
        0x4     UNMAP         .. segment unmapped
        0x8     MUNMAP        .. next segment in the template unmapped
        0x10    REVERSE       .. SEQ is reverse complemented
        0x20    MREVERSE      .. SEQ of the next segment in the template is reversed
        0x40    READ1         .. the first segment in the template
        0x80    READ2         .. the last segment in the template
        0x100   SECONDARY     .. secondary alignment
        0x200   QCFAIL        .. not passing quality controls
        0x400   DUP           .. PCR or optical duplicate
        0x800   SUPPLEMENTARY .. supplementary alignment
```

```
1357452 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
3938 + 0 supplementary
0 + 0 duplicates
1349517 + 0 mapped (99.42% : N/A)
1353514 + 0 paired in sequencing
676757 + 0 read1
676757 + 0 read2
1335324 + 0 properly paired (98.66% : N/A)
1342946 + 0 with itself and mate mapped
2633 + 0 singletons (0.19% : N/A)
5420 + 0 with mate mapped to a different chr
3843 + 0 with mate mapped to a different chr (mapQ>=5)
```

### 5.1.6

1. Viewer to show the alignment to the reference genom in text. Top line shows N's if reference is unknown, underneath is the consensus. Dot is a match for forward strand, comma is a match for reverse strand
2. `samtools tview -p NW_006238923.1:1000000 readmapping.sorted crubellaData/ncbi_dataset/data/idx/ref`
   1. When you see a letter, then 
3. Yes, everytime you see a letter
4. When you go down and move right, then you see whether there are the same amount of lines

### 5.1.7

